In [40]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

In [22]:
# URL du 1er fichier CSV :
url = 'https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv'

# Lire le fichier CSV et créer  DataFrame :
df = pd.read_csv(url)

df.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [23]:
df.shape

(800, 12)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [25]:
df.describe()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.0000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,400.5000,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,231.0844,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.0000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,200.7500,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,400.5000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,600.2500,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,800.0000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [26]:
# Sépare les Pokémon légdaires (Legendary == True) et non-légdaires (Legendary == False) :
legendary_pokemon = df[df['Legendary'] == True]
non_legendary_pokemon = df[df['Legendary'] == False]

In [27]:
legendary_pokemon

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
156,157,Articuno,Ice,Flying,90,85,100,95,125,85,1,True
157,158,Zapdos,Electric,Flying,90,90,85,125,90,100,1,True
158,159,Moltres,Fire,Flying,90,100,90,125,85,90,1,True
162,163,Mewtwo,Psychic,NaN,106,110,90,154,90,130,1,True
163,164,Mega Mewtwo X,Psychic,Fighting,106,190,100,154,100,130,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


In [28]:
non_legendary_pokemon

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
787,788,Gourgeist Super Size,Ghost,Grass,85,100,122,58,75,54,6,False
788,789,Bergmite,Ice,NaN,55,69,85,32,35,28,6,False
789,790,Avalugg,Ice,NaN,95,117,184,44,46,28,6,False
790,791,Noibat,Flying,Dragon,40,30,35,45,40,55,6,False


In [29]:
# Sélectionne uniquemt les colonnes numériq :
numeric_columns = df.select_dtypes(include='number')

# Instancier le StandardScaler :
scaler = StandardScaler()

# Appliq le scaler sur les données numériq et mise à l'échelle :
df_scaled = numeric_columns.copy()
df_scaled = pd.DataFrame(scaler.fit_transform(df_scaled), columns=numeric_columns.columns)

df_scaled

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,-1.729887,-0.950626,-0.924906,-0.797154,-0.239130,-0.248189,-0.801503,-1.399637
1,-1.725557,-0.362822,-0.524130,-0.347917,0.219560,0.291156,-0.285015,-1.399637
2,-1.721227,0.420917,0.092448,0.293849,0.831146,1.010283,0.403635,-1.399637
3,-1.716897,0.420917,0.647369,1.577381,1.503891,1.729409,0.403635,-1.399637
4,-1.712567,-1.185748,-0.832419,-0.989683,-0.392027,-0.787533,-0.112853,-1.399637
...,...,...,...,...,...,...,...,...
795,1.712567,-0.754692,0.647369,2.443765,0.831146,2.808099,-0.629341,1.611954
796,1.716897,-0.754692,2.497104,1.160233,2.665905,1.369846,1.436611,1.611954
797,1.721227,0.420917,0.955658,-0.444182,2.360112,2.088973,0.059310,1.611954
798,1.725557,0.420917,2.497104,-0.444182,2.971699,2.088973,0.403635,1.611954


In [30]:
# Filtre les Pokémon non-légdaires :
df_non_legendary = df[df['Legendary'] == False]

# Sélectionne uniquemt les colonnes numériq :
numeric_columns = df_non_legendary.select_dtypes(include='number')

# Nrmalise les données numériq :
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numeric_columns)

# Instancie et entraîne le mdèle NearestNeighbors :
# On ajuste le nb de voisins
model = NearestNeighbors(n_neighbors=5) 
model.fit(X_scaled)

# Affiche 1 exple de voisins pr le 1er Pokémon non-légdaire :
distances, indices = model.kneighbors([X_scaled[0]])

# Affiche les indices et les distces des voisins :
print(f"Indices des voisins : {indices}")
print(f"Distces aux voisins : {distances}")

Indices des voisins : [[ 0 48  9 59 90]]
Distces aux voisins : [[0.         0.70151611 0.74455308 0.76661174 0.9530515 ]]


In [37]:
# Sélectionne les Pokémon légdaires et non-légdaires :
df_legendary = df[df['Legendary'] == True]
df_non_legendary = df[df['Legendary'] == False]

# Sélectionne les colonnes numériq :
numeric_columns = df.select_dtypes(include='number')

# Nrmalise les données des Pokémon légdaires et non-légdaires :
scaler = StandardScaler()
X_legendary_scaled = scaler.fit_transform(df_legendary[numeric_columns.columns])
X_non_legendary_scaled = scaler.transform(df_non_legendary[numeric_columns.columns])

# Instancie et entraîne le mdèle NearestNeighbors :
# On ajuste le nb de voisins
model = NearestNeighbors(n_neighbors=4)
model.fit(X_non_legendary_scaled)

# Pr chaq Pokémon légdaire, trve les k voisins non-légdaires les + proches :
for idx, legendary_pokemon in df_legendary.iterrows():
    # Utilise le bon indice pr X_legendary_scaled
    legendary_data_scaled = X_legendary_scaled[df_legendary.index.get_loc(idx)].reshape(1, -1)

    # Trve les voisins les + proches :
    distances, indices = model.kneighbors(legendary_data_scaled)
    
    # Affiche le nom du Pokémon légdaire et les voisins non-légdaires les + proches :
    print(f"Pokémon légdaire: {legendary_pokemon['Name']}")
    print("Voisins non-légdaires les + proches:")
    
    # Affiche les 4 voisins les + proches :
    for i in range(4):  
        neighbor_idx = indices[0][i]
        
        # S'Assure q le voisin trvé n'est pas 1 Pokémon légdaire :
        if df_non_legendary.iloc[neighbor_idx]['Legendary'] == False:
            neighbor_name = df_non_legendary.iloc[neighbor_idx]['Name']
            print(f"  - {neighbor_name} (Distance: {distances[0][i]:.2f})")
    
    print("\n")

Pokémon légdaire: Articuno
Voisins non-légdaires les + proches:
  - Blastoise (Distance: 1.24)
  - Meganium (Distance: 1.29)
  - Mew (Distance: 1.30)
  - Venusaur (Distance: 1.44)


Pokémon légdaire: Zapdos
Voisins non-légdaires les + proches:
  - Typhlosion (Distance: 1.09)
  - Charizard (Distance: 1.16)
  - Mew (Distance: 1.18)
  - Arcanine (Distance: 1.24)


Pokémon légdaire: Moltres
Voisins non-légdaires les + proches:
  - Arcanine (Distance: 1.10)
  - Mew (Distance: 1.21)
  - Typhlosion (Distance: 1.31)
  - Charizard (Distance: 1.37)


Pokémon légdaire: Mewtwo
Voisins non-légdaires les + proches:
  - Mega Pidgeot (Distance: 1.86)
  - Mega Houndoom (Distance: 1.96)
  - Mega Charizard Y (Distance: 2.26)
  - Mew (Distance: 2.29)


Pokémon légdaire: Mega Mewtwo X
Voisins non-légdaires les + proches:
  - Mega Blaziken (Distance: 2.84)
  - Mega Salamence (Distance: 2.96)
  - Mega Charizard X (Distance: 3.02)
  - Dragonite (Distance: 3.44)


Pokémon légdaire: Mega Mewtwo Y
Voisins non-lé

In [39]:
# Sélectionne les Pokémon légdaires et non-légdaires :
df_legendary = df[df['Legendary'] == True]
df_non_legendary = df[df['Legendary'] == False]

# Sélectionne les colonnes numériq :
numeric_columns = df.select_dtypes(include='number')

# Nrmalise les données des Pokémon légdaires et non-légdaires :
scaler = StandardScaler()
X_legendary_scaled = scaler.fit_transform(df_legendary[numeric_columns.columns])
X_non_legendary_scaled = scaler.transform(df_non_legendary[numeric_columns.columns])

# Instancie et entraîne le mdèle NearestNeighbors :
# On ajuste le nb de voisins :
model = NearestNeighbors(n_neighbors=4)
model.fit(X_non_legendary_scaled)

# Pr chaq Pokémon légdaire, trve les k voisins non-légdaires les + proches :
for idx, legendary_pokemon in df_legendary.iterrows():
    # Utilise le bon indice pr X_legendary_scaled :
    legendary_data_scaled = X_legendary_scaled[df_legendary.index.get_loc(idx)].reshape(1, -1)

    # Trve les voisins les + proches :
    distances, indices = model.kneighbors(legendary_data_scaled)
    
    # Affiche le nom du Pokémon légdaire et les voisins non-légdaires les + proches :
    print(f"Pokémon légendaire: {legendary_pokemon['Name']}")
    
    # Créeait 1 liste de voisins avec leur distce et caractéristiq :
    recommended_pokemon = []
    # Affiche les 4 voisins les + proches :
    for i in range(4):  
        neighbor_idx = indices[0][i]
        
        # S'Assure q le voisin trvé n'est pas 1 Pokémon légdaire :
        if df_non_legendary.iloc[neighbor_idx]['Legendary'] == False:
            neighbor_name = df_non_legendary.iloc[neighbor_idx]['Name']
            neighbor_distance = distances[0][i]
            
            # Extraire les caractéristiq pertinentes :
            neighbor_data = df_non_legendary.iloc[neighbor_idx][['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]
            recommended_pokemon.append((neighbor_name, neighbor_distance, neighbor_data))
    
    # Tri les voisins par la distce (similitude) :
    # Tri par distce croissante (meilleure similarité) :
    recommended_pokemon.sort(key=lambda x: x[1])  
    
    # Affiche des voisins recommandés triés par similarité :
    print("Recommandat° (par ordre de similarité):")
    for neighbor in recommended_pokemon:
        print(f"  - {neighbor[0]} (Distance: {neighbor[1]:.2f})")
        print(f"    - HP: {neighbor[2]['HP']}, Attack: {neighbor[2]['Attack']}, Defense: {neighbor[2]['Defense']}")
        print(f"    - Sp. Atk: {neighbor[2]['Sp. Atk']}, Sp. Def: {neighbor[2]['Sp. Def']}, Speed: {neighbor[2]['Speed']}")
    print("\n")

Pokémon légendaire: Articuno
Recommandat° (par ordre de similarité):
  - Blastoise (Distance: 1.24)
    - HP: 79, Attack: 83, Defense: 100
    - Sp. Atk: 85, Sp. Def: 105, Speed: 78
  - Meganium (Distance: 1.29)
    - HP: 80, Attack: 82, Defense: 100
    - Sp. Atk: 83, Sp. Def: 100, Speed: 80
  - Mew (Distance: 1.30)
    - HP: 100, Attack: 100, Defense: 100
    - Sp. Atk: 100, Sp. Def: 100, Speed: 100
  - Venusaur (Distance: 1.44)
    - HP: 80, Attack: 82, Defense: 83
    - Sp. Atk: 100, Sp. Def: 100, Speed: 80


Pokémon légendaire: Zapdos
Recommandat° (par ordre de similarité):
  - Typhlosion (Distance: 1.09)
    - HP: 78, Attack: 84, Defense: 78
    - Sp. Atk: 109, Sp. Def: 85, Speed: 100
  - Charizard (Distance: 1.16)
    - HP: 78, Attack: 84, Defense: 78
    - Sp. Atk: 109, Sp. Def: 85, Speed: 100
  - Mew (Distance: 1.18)
    - HP: 100, Attack: 100, Defense: 100
    - Sp. Atk: 100, Sp. Def: 100, Speed: 100
  - Arcanine (Distance: 1.24)
    - HP: 90, Attack: 110, Defense: 80
    - S

Les recommandat° faites par le mdèle NearestNeighbors pr chaq Pokémon légendaire pvent-ê pertinentes ds 1 ctexte où l'on cherche à cparer les Pokémon en fct° de leurs statistiq de combat. Cpdt, plsrs améliorat° st possibles :

L'inclus° des types des Pokémon et d'autres caractéristiq comme leurs cpacités spéciales et leur rôle stratégiq ds les combats permettrait d'améliorer la pertinence des recommandat°.
Le choix de k a 1 impact imprtant sur la qualité des résultats et devrait ê ajusté en fct° des objs. Il est recommandé d'expérimenter avec différentes vleurs de k et de mesurer leur impact sur la précis° des recommandat°.